In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

In [ ]:
# Let's load a sample dataset with 82k cells from a flow cytometry experiment.

import pickle
import numpy as np

tds = pickle.load(open('flow-sample.pkl', "rb"))
X = tds['X'] # the raw values (approximately normally distributed)
Y = tds['Y'] # cell-type labels for reference
markers = tds['markers'] # meaning of columns (not used)

In [ ]:
X.shape

In [ ]:
# Prepare a function for making scatter plots of latent space embeddings

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

classes = sorted(list(set(Y)))
colormap = plt.cm.hsv
class_colors = [colormap(float(ci)/len(classes)) for ci in range(len(classes))]
legend_handles = [mpatches.Patch(color=class_colors[ci], label=classes[ci]) for ci in range(len(classes))]
colors = [class_colors[classes.index(y)] for y in Y] # colour vector corresponding to Y

def plot(Z, title):
    plt.figure(figsize=(8, 8))
    plt.scatter(Z[:, 0], Z[:, 1], c=colors, s=0.1)
    plt.legend(handles=legend_handles)
    plt.xlabel('dim 1')
    plt.ylabel('dim 2')
    plt.title(title)
    plt.show()

In [ ]:
# TruncatedSVD implements PCA without the need to center the data
# and only calculates the number of dimensions you specify (e.g. n_components=2).

from sklearn.decomposition import TruncatedSVD

pca = TruncatedSVD(n_components=2)
Z_pca = pca.fit_transform(X)

In [ ]:
plot(Z_pca, 'Cell types - PCA (first 2 components)')

In [ ]:
# Using Keras to make an autoencoder

import tensorflow.keras as keras
from tensorflow.keras.layers import *

In [ ]:
# The simplest autoencoder has one linear layer for encoding, and one linear layer for decoding.
# It's worth using the explicit Functional API to keep references to different sub-parts of the network.

# Instantiate the layer tensors:

inp = Input(shape=(X.shape[1],)) # input for X
enc = Dense(units=2, activation='linear') # for encoding to Z
dec = Dense(units=X.shape[1], activation='linear') # for decoding back to X

In [ ]:
# Connecting layers is done through nested function call syntax on tensors:

reconstruction = dec(enc(inp))

In [ ]:
# Create Keras Model object from connected layers to be able to predict from the stack.

linear_AE = keras.Model(inputs=inp, outputs=reconstruction)
linear_AE.summary()

In [ ]:
# Simple training in Keras using SGD optimiser and MSE reconstruction loss:

linear_AE.compile(optimizer='sgd', loss='mse')
linear_AE.fit(X, X, epochs=10, batch_size=32, verbose=2)

# (change number of epochs and batch size if you're not satisfied with the loss convergence)

In [ ]:
# To predict Z from X we want a model from the first input to the output of encoder, using existing layers:

linear_encoder = keras.Model(inp, enc(inp))

In [ ]:
# Call predict to get latent embedding in Z:

Z_ae_lin = linear_encoder.predict(X)

In [ ]:
plot(Z_ae_lin, 'Cell types - linear autoencoder (2D)')

In [ ]:
# A deep autoencoder with two hidden layers in both encoder and decoder.
# You can experiment with the number of hidden layers, number of units, types of hidden activation.
# Because there are layers which we don't need to directly access afterwards,
# we can stack them right away and only keep references to important endpoints:

inp = Input(shape=(X.shape[1],))
hidden = Dense(units=128, activation='elu')(inp)
hidden = Dense(units=128, activation='elu')(hidden)
enc = Dense(units=2, activation='linear')(hidden)
hidden = Dense(units=128, activation='elu')(enc)
hidden = Dense(units=128, activation='elu')(hidden)
reconstruction = Dense(units=X.shape[1], activation='linear')(hidden)

In [ ]:
AE = keras.Model(inp, reconstruction)
AE.summary()

In [ ]:
AE.compile(optimizer='sgd', loss='mse')
AE.fit(X, X, epochs=10, batch_size=32, verbose=2)

In [ ]:
encoder = keras.Model(inp, enc)
Z_ae = encoder.predict(X)

In [ ]:
plot(Z_ae, 'Cell types - deep autoencoder (2D)')

In [ ]:
# VAE
import tensorflow as tf

# This custom layer returns a random sample from a normal distribution of provided mean and sd.
# Log variance is input for numerical stability and turned to standard deviation.

class Sampling(keras.layers.Layer):
    def call(self, inputs):
        # assuming two inputs, one for mean, one for log variance
        z_mean, z_log_var = inputs
        # sample from unit Gaussian:
        epsilon = keras.backend.random_normal(shape=tf.shape(z_mean))
        # reparameterise to mean + sd * N(0, 1)
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
inp = Input(shape=(X.shape[1],))
hidden = Dense(units=128, activation='elu')(inp)
hidden = Dense(units=128, activation='elu')(hidden)

# Here we have two parallel layers, first predicting mean, second predicting log variance for each latent dimension
enc_mean = Dense(units=2, activation='linear')(hidden)
enc_log_var = Dense(units=2, activation='linear')(hidden)
# and the sampling layer which turns the two parameters into a single probabilistic value in Z.
enc = Sampling()([enc_mean, enc_log_var])

# decoder takes the probabilistic output and projects it back to data space:
hidden = Dense(units=128, activation='elu')(enc)
hidden = Dense(units=128, activation='elu')(hidden)
reconstruction = Dense(units=X.shape[1], activation='linear')(hidden)

In [ ]:
VAE = keras.Model(inp, reconstruction)
VAE.summary()

In [ ]:
# Losses are also tensors and can be created explicitly.
# Create the two loss components that sum in the ELBO.

# Reconstruction term:
rec_loss = tf.reduce_mean(keras.losses.mse(inp, reconstruction))

# and the KL divergence regularisation term:
kl_loss = -0.5 * tf.reduce_mean(enc_log_var - tf.square(enc_mean) - tf.exp(enc_log_var) + 1)

# Adjust the amount of regularisation by weighing the terms:
vae_loss = rec_loss + 0.3 * kl_loss

VAE.add_loss(vae_loss)

In [ ]:
VAE.compile(optimizer='sgd')
VAE.fit(X, X, epochs=10, batch_size=32, verbose=2)

In [ ]:
# Get the encoder and predict the probabilistic latent embedding in Z:

encoder = keras.Model(inp, enc)
Z_vae = encoder.predict(X)

In [ ]:
plot(Z_vae, 'Cell types - deep VAE (2D)')

In [ ]:
# We can also have a look at the deterministic mean output:

encoder_mean = keras.Model(inp, enc_mean)
Z_mean = encoder_mean.predict(X)

In [ ]:
plot(Z_mean, 'Cell types - deep VAE (2D) - mean')